# AutoGen Application Function Calling

This notebook shows an example of building AutoGen application for currency exchange, using the LLM model from OCI Generative AI service.

This notebook is for AutoGen v0.2
```
pip install autogen-agentchat~=0.2
```

This notebook is based on the example:
https://microsoft.github.io/autogen/0.2/docs/notebooks/agentchat_function_call_currency_calculator

This notebook requires **Python 3.9** or newer. Please also install the latest version of Oracle ADS:
```
pip install oracle-ads
```

The following dependencies are also required:
```
pip install "langchain-community>0.3" langchain-openai report_creator
```

## Custom LLM Client for AutoGen

ADS provides the custom LLM client for AutoGen to enable the use of LangChain chat models, including the [ChatOCIGenAI](https://python.langchain.com/docs/integrations/chat/oci_generative_ai/) and the [ChatOCIModelDeployment](https://python.langchain.com/docs/integrations/chat/oci_data_science/).

When using custom client with AutoGen, you need to register the custom client with each agent. To simplify the process, ADS provides a `register_custom_client` function to register the custom client globally. With this function, you only need to register the client once. This will allow you to easily switch between OpenAI model and custom model in the LLM config without any code change or if/else logic.

In [ ]:
# Register the custom LLM globally so that we don't need to do it for each agent
from ads.llm.autogen.v02 import LangChainModelClient, register_custom_client
 
register_custom_client(LangChainModelClient)

## LLM Config

Following is an example of the LLM config for the OCI Generative AI service.
The `langchain_cls` is the full path for importing the LangChain chat model, while the `client_params` contains the arguments for initializing the LangChain chat model.
```
gen_ai_config = [
    {
        "model_client_cls": "LangChainModelClient",
        "langchain_cls": "langchain_community.chat_models.oci_generative_ai.ChatOCIGenAI",
        "model": "cohere.command-r-plus",
        # client_params will be used to initialize the LangChain ChatOCIGenAI class.
        "client_params": {
            "model_id": "cohere.command-r-plus",
            "compartment_id": COMPARTMENT_OCID,
            "model_kwargs": {"temperature": 0, "max_tokens": 2048},
            "auth_type": "SECURITY_TOKEN",
            "auth_profile": "<PROFILE_NAME>",
        },
    }
]
```

In [ ]:

gen_ai_config = [
    {
        "model_client_cls": "LangChainModelClient",
        "langchain_cls": "langchain_community.chat_models.oci_generative_ai.ChatOCIGenAI",
        "model": "cohere.command-r-plus",
        # client_params will be used to initialize the LangChain ChatOCIGenAI class.
        "client_params": {
            "model_id": "cohere.command-r-plus-08-2024",
            "compartment_id": "<COMPARTMENT_OCID>",
            "service_endpoint": "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
            "model_kwargs": {"temperature": 0, "max_tokens": 4000},
            # Set auth_type to INSTANCE_PRINCIPAL or RESOURCE_PRINCIPAL
            "auth_type": "SECURITY_TOKEN",
            # auth_profile is needed only if you are using API_KEY or SECURITY_TOKEN
            "auth_profile": "<PROFILE_NAME>",
        },
    }
]

# AutoGen LLM config
llm_config = {
    # To try other model, simply use a different config
    "config_list": gen_ai_config,
    # Disable cache
    # https://microsoft.github.io/autogen/0.2/docs/topics/llm-caching/
    "cache_seed": None,
}

## Create the Agents with LLM Config

The following code creates two agents. Note that there is no need to register the custom LLM.

In [5]:
import autogen

chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message=(
        "For currency exchange rates, only use the functions you have been provided with. "
        "Calculate the currency exchange based on the rates returned by the function. "
        "Reply TERMINATE when the task is done."
    ),
    llm_config=llm_config,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config=False,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
)

## Define the Tools (Functions)

The following code defines two tool. One for getting the currency exchange rate and the other for calculating the currency based on the exchange rate.

In [6]:
import time
from typing import Annotated
import requests


@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Obtain exchange rate from one currency to other currencies.")
def get_exchange_rate(
        currency: Annotated[str, "currency in ISO 4217 3-letter currency code"]
    ) -> Annotated[dict, "A dictionary containing the exchange rate to other currencies"]:
    """Obtain the current exchange rates of a currency in ISO 4217 3-letter currency code"""
    response = requests.get(f"https://open.er-api.com/v6/latest/{currency}")
    # Adding two seconds delay here to simulate a tool call that is running a bit longer.
    time.sleep(2)
    return response.json()

@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Currency exchange calculator.")
def currency_calculator(
        base_amount: Annotated[float, "Amount of currency to be exchanged."],
        exchange_rate: Annotated[float, "Exchange rate."],
    ) -> Annotated[float, "Amount of currency exchanged into."]:
    """Calculates the currency exchange"""
    return base_amount * exchange_rate

## Start the AutoGen Application

Now everything is defined, we can start running the AutoGen application.

In [7]:
res = user_proxy.initiate_chat(
    chatbot, message="How much is 123.45 USD in EUR?", summary_method="reflection_with_llm"
)

user_proxy (to chatbot):

How much is 123.45 USD in EUR?

--------------------------------------------------------------------------------
chatbot (to user_proxy):

I will first find the exchange rate from USD to EUR. Then, I will use the currency calculator to find out how much 123.45 USD is in EUR.
***** Suggested tool call (b5657fd24d9d42f9899e3b441575f4e6): get_exchange_rate *****
Arguments: 
{"currency": "USD"}
*************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_exchange_rate...
user_proxy (to chatbot):

user_proxy (to chatbot):

***** Response from calling tool (b5657fd24d9d42f9899e3b441575f4e6) *****
{"result": "success", "provider": "https://www.exchangerate-api.com", "documentation": "https://www.exchangerate-api.com/docs/free", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1744502552, "time_la